In [2]:
# Install TensorFlow if it is not already installed
!pip install tensorflow

# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/376.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/376.9 MB 10.9 MB/s eta 0:00:35
   ---------------------------------------- 1.6/376.9 MB 16.4 MB/s eta 0:00:23
   ---------------------------------------- 3.0/376.9 MB 24.2 MB/s eta 0:00:16
    --------------------------------------- 5.5/376.9 MB 29.6 MB/s eta 0:00:13
    --------------------------------------- 7.5/376.9 MB 34.4 MB/s eta 0:00:11
   - -------------------------------------- 9.5/376.9 MB 35.7 MB/s eta 0:00:11
   - -------------------------------------- 12.1/376.9 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 14.4/376.9 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 16.6/376.9 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 18.3/376.9 MB 46.7 MB/s et

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [4]:
# Bin the data to optimize the model
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < 500].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

application_df = pd.get_dummies(application_df)
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns=['IS_SUCCESSFUL'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the optimized model
nn_optimized = tf.keras.models.Sequential()

# Add an input layer
nn_optimized.add(tf.keras.layers.InputLayer(shape=(X_train.shape[1],)))

# First hidden layer with increased units
nn_optimized.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Add a third hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Second hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_optimized.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 100)                 │           4,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,851 (42.39 KB)

 Trainable params: 10,851 (42.39 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the optimized model
nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the optimized model
history_optimized = nn_optimized.fit(X_train_scaled, y_train, epochs=100, verbose=1)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7137 - loss: 0.5812
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7312 - loss: 0.5562
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7350 - loss: 0.5474
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7365 - loss: 0.5451
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7326 - loss: 0.5445
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7292 - loss: 0.5481
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7352 - loss: 0.5471
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7344 - loss: 0.5424
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7316 - loss: 0.5447
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7374 - loss: 0.5418
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7359 - loss: 0.5439
Epoch 12/100
858/858 ━━━━━━━━━━━━━━━━━━━━

In [15]:
# Evaluate the optimized model using the test data
model_loss_optimized, model_accuracy_optimized = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss: {model_loss_optimized}, Optimized Accuracy: {model_accuracy_optimized}")

215/215 - 1s - 3ms/step - accuracy: 0.7255 - loss: 0.5752
Optimized Loss: 0.5752280354499817, Optimized Accuracy: 0.7255101799964905


In [17]:
# Save the optimized model to an HDF5 file
nn_optimized.save("AlphabetSoupCharity_Optimization.keras")